<a href="https://colab.research.google.com/github/spentaur/DS-Unit-2-Applied-Modeling/blob/master/Copy_of_Unit2ProjectFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load in some packages


*   [ta](https://github.com/bukosabino/ta)
*   [eli5](https://eli5.readthedocs.io/en/latest/overview.html)
*   [bayesian-optimization](https://github.com/fmfn/BayesianOptimization)
*   [LightGBM](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html)
*   [h2o4gpu](https://github.com/h2oai/h2o4gpu)

In [1]:
%%sh
pip install ta
pip install eli5
pip install bayesian-optimization
pip install pandas_market_calendars
pip install pytz

# LightGBM
# git clone --recursive https://github.com/microsoft/LightGBM ; cd LightGBM
# mkdir build ; cd build
# cmake -DUSE_GPU=1 ..
# make -j4

# h2o4gpu
export CUDA_HOME=/usr/local/cuda 
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CUDA_HOME/lib64/:$CUDA_HOME/lib/:$CUDA_HOME/extras/CUPTI/lib64
sudo apt-get install libopenblas-dev pbzip2
wget https://s3.amazonaws.com/h2o-release/h2o4gpu/releases/stable/ai/h2o/h2o4gpu/0.3-cuda10/h2o4gpu-0.3.2-cp36-cp36m-linux_x86_64.whl
pip install h2o4gpu-0.3.2-cp36-cp36m-linux_x86_64.whl

  Created wheel for ta: filename=ta-0.4.7-cp36-none-any.whl size=18553 sha256=031c4712a146db54b18682fd68ae6457361cf294e20c8d316ae0f0281bb8ef1c
  Stored in directory: /root/.cache/pip/wheels/fe/58/16/b989347aa65ea8047524fdf38b5e70e2605e58eb8e46396368
Successfully built ta
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=b5fcc643932067396b608025e761dd96cddd16924f8add25b43808ff131fa1dd
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization
Reading package lists...
Building dependency tree...
Reading state information...
libopenblas-dev is already the newest version (0.2.20+ds-4).
The following NEW packages will be installed:
  pbzip2
0 upgraded, 1 newly installed, 0 to remove and 28 not upgraded.
Need to get 37.1 kB of archives.
After this operation, 95.2 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubu

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
--2019-10-23 03:40:00--  https://s3.amazonaws.com/h2o-release/h2o4gpu/releases/stable/ai/h2o/h2o4gpu/0.3-cuda10/h2o4gpu-0.3.2-cp36-cp36m-linux_x86_64.whl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.240.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.240.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 366174349 (349M) [application/zip]
Saving to: ‘h2o4gpu-0.3.2-cp36-cp36m-linux_x86_64.whl’

     0K .......... .......... .......... .......... ..........  0%  784K 7m36s
    50K .......... .......

In [5]:
pip install pytz --upgrade

Requirement already up-to-date: pytz in /usr/local/lib/python3.6/dist-packages (2019.3)


In [8]:
pip install pytz

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import scipy.stats as st
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline
from sklearn.model_selection import train_test_split
from ta import *
from scipy.stats import ttest_ind
from scipy import stats
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.feature_selection import SelectFromModel
import lightgbm as lgb
from sklearn.metrics import auc, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from scipy import stats
import pandas_datareader.data as web
from datetime import datetime
import h2o4gpu
from bayes_opt import BayesianOptimization
from sklearn.feature_selection import SelectFromModel
from dateutil.rrule import *
import pandas_market_calendars as mcal
import time
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Change some pandas defaults

In [0]:
def start():
    options = {
        'display': {
            'max_columns': None,
            'max_colwidth': 25,
            'expand_frame_repr': False,  # Don't wrap to multiple pages
            'max_rows': 100,
            'max_seq_items': 50,         # Max length of printed sequence
            'precision': 4,
            'show_dimensions': False
        },
        'mode': {
            'chained_assignment': None   # Controls SettingWithCopyWarning
        }
    }
    for category, option in options.items():
        for op, value in option.items():
            pd.set_option(f'{category}.{op}', value)  # Python 3.6+
start()

# Load in historical data for the SPY ETF

In [11]:
spy = web.DataReader("SPY", 'yahoo').reset_index()
spy.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2010-01-04,113.39,111.51,112.37,113.33,1.1894e+08,93.2447
1,2010-01-05,113.68,112.85,113.26,113.63,1.1158e+08,93.4915
2,2010-01-06,113.99,113.43,113.52,113.71,1.1607e+08,93.5573
3,2010-01-07,114.33,113.18,113.50,114.19,1.3109e+08,93.9523
4,2010-01-08,114.62,113.66,113.89,114.57,1.2640e+08,94.2649


In [12]:
spy.tail()

,Date,High,Low,Open,Close,Volume,Adj Close
2463,2019-10-16,299.16,297.92,298.37,298.40,4.8928e+07,298.40
2464,2019-10-17,300.24,298.52,299.68,299.28,4.5737e+07,299.28
2465,2019-10-18,299.40,297.03,298.69,297.97,6.4304e+07,297.97
2466,2019-10-21,300.21,298.94,299.42,299.99,3.9049e+07,299.99
2467,2019-10-22,300.90,298.91,300.58,299.01,4.8595e+07,299.01


# get target

In [0]:
def get_target(data, days_out):
    data = data.copy()
    # get the close price X days out
    data['x_close'] = data['Close'].shift(days_out)
    # get the % return 20 days out
    data['x_return'] = (((data['x_close'] / data['Close']) - 1)  * 100)
    # is return greater than 1%
    data['Up'] = data['x_return'] > 1
    # price at 1% return
    data['1% Price'] = data['Close'] * 1.01
    # get the date 20 days out
    nyse = mcal.get_calendar('NYSE')
    nyse_schedule = nyse.schedule(start_date='2010-01-04', end_date='2022-12-31').reset_index()
    nyse_schedule.rename(columns={'index': 'Date'}, inplace=True)
    nyse_schedule = spy.merge(nyse_schedule, how = "outer")
    data['x_days_out'] = nyse_schedule['Date'].shift(days_out)

    return data[['Date','Close', 'x_close', 'x_return', '1% Price', 'Up', 'x_days_out']]

In [14]:
days_out = -20
target = get_target(spy, days_out)
print(target.shape)
target.tail(30)

(2468, 7)


,Date,Close,x_close,x_return,1% Price,Up,x_days_out
2438,2019-09-11,300.25,291.27,-2.9908,303.2525,False,2019-10-09
2439,2019-09-12,301.29,293.24,-2.6719,304.3029,False,2019-10-10
2440,2019-09-13,301.09,296.28,-1.5975,304.1009,False,2019-10-11
2441,2019-09-16,300.16,295.95,-1.4026,303.1616,False,2019-10-14
2442,2019-09-17,300.92,298.88,-0.6779,303.9292,False,2019-10-15
2443,2019-09-18,301.10,298.40,-0.8967,304.1110,False,2019-10-16
2444,2019-09-19,301.08,299.28,-0.5978,304.0908,False,2019-10-17
2445,2019-09-20,298.28,297.97,-0.1039,301.2628,False,2019-10-18
2446,2019-09-23,298.21,299.99,0.5969,301.1921,False,2019-10-21
2447,2019-09-24,295.87,299.01,1.0613,298.8287,True,2019-10-22


# Wrangle

In [0]:
def wrangle(data, drop_date=True):
    data = data.copy()

    # change Date to datetime type
    data['Date'] = pd.to_datetime(data['Date'])
    # change date to index
    data.index = pd.DatetimeIndex(data['Date'])

    # not sure if this is needed, but since i'm doing just a basic train test
    # split i want to make sure it's sorted
    data = data.sort_index()

    # add some time features
    data['Day of Week'] = data['Date'].dt.weekday
    data['Year'] = data['Date'].dt.year
    data['Month'] = data['Date'].dt.month
    data['Quarter'] = data['Date'].dt.quarter
    data['Week'] = data['Date'].dt.quarter

    # drop the actual date column

    data = data.drop('Adj Close', axis=1)

    if drop_date:
        data = data.drop('Date', axis=1)
        
    # Daily High - Daily Low
    data['High - Low'] = data['High'] - data['Low']
    data['High - Low as % of Close'] = data['High - Low'] / data['Close']
    # High - Close
    data['High - Close'] = data['High'] - data['Close']
    data['High - Close as % of Close'] = data['High - Close'] / data['Close']
    # High - Open
    data['High - Open'] = data['High'] - data['Open']
    data['High - Open as % of Close'] = data['High - Open'] / data['Close']
    # Today Open - Yesterday Close
    data['Open - Close'] = data['Open'] - data['Close'].shift(1)
    data['Open - Close as % of Close'] = data['Open - Close'] / data['Close']


    # add all the technical analysis features from ta
    data = add_all_ta_features(data, "Open", "High", "Low", "Close", 
                               "Volume", fillna=False)
    
    # delete the columns that are over 50% missing
    perc_missing = data.isna().sum() / len(data)
    data = data.drop(perc_missing[perc_missing > .50].index.tolist(), axis=1)

    # some of the features use ~45 day rolling averages and stuff
    # so i'll just skip the first 45 days
    # and then i'll also remove the days that wont have the target
    # since it hasn't happened yet (20 days out)
    data = data.iloc[45:]

    # i'm just going to divide every feature by every other feature
    # there's gotta be a pattern in the data somewhere right
    div_features = pd.DataFrame()
    cols = data.select_dtypes('number').columns.tolist()
    n = len(cols)
    for idx, first in enumerate(cols):
        x = (n-idx) - 1
        # for second in cols[:x]:
        for second in cols:
            div_features['{} / {}'.format(first, second)] = data[first] / data[second]

    data = pd.concat([data, div_features], axis=1)
    
    # now i gotta do something about all the inf values i just created
    data = data.replace([np.inf, -np.inf], np.nan)
    data = data.interpolate(method='time')
    data = data.fillna(data.mean())

    perc_missing = data.isna().sum() / len(data)
    data = data.drop(perc_missing[perc_missing > .50].index.tolist(), axis=1)

    nunique = data.nunique()
    data = data.drop(nunique[nunique == 1].index.tolist(), axis=1)
    
    # done
    return data

In [16]:
# wrangle only the data for which i'd have the target
df = wrangle(spy.iloc[:days_out])
df.shape

/usr/local/lib/python3.6/dist-packages/ta/trend.py:170: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:174: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])


(2403, 5615)

# Fit the model

In [17]:
target_feature = 'Up'
X = df
y = target[target_feature].iloc[45:days_out]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42, shuffle=False)
X_train.shape, X_test.shape

((2282, 5615), (121, 5615))

In [0]:
clf = ''
clf = h2o4gpu.GradientBoostingClassifier(n_estimators=800,n_jobs=-1, random_state=42)

clf.fit(X_train, y_train)
ypred = clf.predict(X_test)
accuracy_score(y_test, ypred)
# 0.6859504132231405

In [0]:
y_test.value_counts(normalize=True)

# Select Best Features

In [79]:
target_feature = 'Up'
X = df
y = target[target_feature].iloc[45:days_out]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.00001, random_state=42, shuffle=False)
X_train.shape, X_test.shape

((2402, 5615), (1, 5615))

In [0]:
clf = ''
clf = h2o4gpu.GradientBoostingClassifier(n_estimators=800,n_jobs=-1, random_state=42)

clf.fit(X_train, y_train)
ypred = clf.predict(X_test)

In [85]:
ypred

array(False)

In [86]:
y_test.value_counts(normalize=True)

True    1.0
Name: Up, dtype: float64

In [87]:
features = pd.DataFrame({'col': X_train.columns, 'importance': clf.feature_importances_}).sort_values('importance', ascending=False)
features.head(30)

,col,importance
2107,volatility_bbl / vola...,0.0217
4677,trend_aroon_ind / vol...,0.0203
5170,momentum_stoch_signal...,0.0131
3403,trend_adx_neg / Quarter,0.0129
1971,volatility_atr / tren...,0.0126
3654,trend_vortex_diff / v...,0.0121
1542,volume_obv / momentum...,0.0103
3608,trend_vortex_ind_neg ...,0.0103
1360,Open - Close as % of ...,0.0087
4467,trend_visual_ichimoku...,0.0082


In [0]:
feature_names = features['col'].tolist()

In [0]:
feature_names_500 = feature_names[:500]

# let's run through a simulation

In [102]:
spy = web.DataReader("SPY", 'yahoo').reset_index()
target = get_target(spy, days_out).iloc[45:days_out]
spy = wrangle(spy.iloc[:days_out], False)
spy.shape, target.shape

/usr/local/lib/python3.6/dist-packages/ta/trend.py:170: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:174: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])


((2403, 5616), (2403, 7))

In [108]:
features_scores = {}
counter = 0
this_year_pred = []
this_year_true = []
for day in range(184, 0, -1):
    start = time.time()
    counter += 1

    test_day = spy.iloc[len(spy) - day]
    test_target = target.iloc[len(spy) - day][target_feature]

    training_days = spy.iloc[:-day]
    y_train = target.iloc[:-day][target_feature]

    this_year_true.append(test_target)

    clf = ''
    clf = h2o4gpu.GradientBoostingClassifier(n_estimators=800,
                                random_state=42)

    clf.fit(training_days.values, y_train)

    features = pd.DataFrame({'col': training_days.columns, 'importance': clf.feature_importances_}).sort_values('importance', ascending=False)
    feature_names = features['col'].tolist()

    X_train_trans = training_days[feature_names[:10]].values
    X_test_trans = test_day[feature_names[:10]].values.reshape(1,-1)

    clf = ''
    clf = RandomForestClassifier(n_estimators=800,
                                n_jobs=-1,
                                random_state=42)

    clf.fit(X_train_trans, y_train)

    ypred = clf.predict(X_test_trans).reshape(1)
    this_year_pred.append(ypred[0])
    score = accuracy_score(this_year_true, this_year_pred)
    if score < .9:
        break
    print('training max date', training_days['Date'].max())
    print('test date', test_day['Date'])
    print('score', accuracy_score(this_year_true, this_year_pred))
    print('majority class', max(y_train.value_counts(normalize=True)))
    print('probability', clf.predict_proba(X_test_trans))
    print('correct?', ypred[0] == test_target)
    print('y_true', test_target)
    print('y_pred', ypred[0])
    print(counter, '/', 184)
    print('time', time.time() - start)
    print('\n')

training max date 2018-12-31 00:00:00
test date 2019-01-02 00:00:00
score 1.0
majority class 0.5358269490761605
probability [[0.04375 0.95625]]
correct? True
y_true True
y_pred True
1 / 184
time 12.601596355438232


training max date 2019-01-02 00:00:00
test date 2019-01-03 00:00:00
score 1.0
majority class 0.536036036036036
probability [[0.06125 0.93875]]
correct? True
y_true True
y_pred True
2 / 184
time 12.453255653381348


training max date 2019-01-03 00:00:00
test date 2019-01-04 00:00:00
score 1.0
majority class 0.5362449347140927
probability [[0.1625 0.8375]]
correct? True
y_true True
y_pred True
3 / 184
time 12.548917293548584


training max date 2019-01-04 00:00:00
test date 2019-01-07 00:00:00
score 1.0
majority class 0.5364536453645364
probability [[0.1625 0.8375]]
correct? True
y_true True
y_pred True
4 / 184
time 12.847776412963867


training max date 2019-01-07 00:00:00
test date 2019-01-08 00:00:00
score 1.0
majority class 0.5366621682411156
probability [[0.0475 0.9525]]

In [0]:
# 0.9230769230769231 - 10 - xgb
# 0.9184782608695652 - 221

In [107]:
sorted(features_scores)

[0.8918918918918919,
 0.8947368421052632,
 0.8974358974358975,
 0.9103139013452914]